## Cleaning US Dataset

In [ ]:
import pandas as pd
from pathlib import Path

def main():
    # Setting US_df as global to use across code
    global US_df

    # Defining file name and path, and reading the CSV
    file_name = "backup_US_quotes.csv"
    output_path = Path.cwd().parent / "data" / "raw" / file_name  # Obtaining the directory
    US_df = pd.read_csv(output_path)

    # Removing " Year Term" from the "Term Length" column and converting to integer
    US_df["Term Length"] = US_df["Term Length"].str.replace(" Year Term", "").astype(int)

    # Removing commas and "$" sign, then convert to float
    US_df["Premium"] = US_df["Premium"].replace({r"\$": "", ",": ""}, regex=True).astype(float)

    # Defining exchange rate (retrieved at time finished collecting final data)
    exchange_rate = 0.80

    # Overwriting the Premium column with GBP values
    US_df["Premium"] = (US_df["Premium"] * exchange_rate).round(2)
    US_df.rename(columns={"Premium": "Premium (£)"}, inplace=True)

    # Adding "Country" column to differentiate after merging
    US_df["Country"] = "US"


    # Printing the DataFrame
    display(US_df)


main()


,Coverage Amount,Term Length,Age,Gender,Is_Smoker,Premium (£),Country
0,100000,10,20,Male,Never Used,7.36,US
1,100000,10,20,Male,Never Used,7.95,US
2,100000,10,20,Male,Never Used,11.27,US
3,100000,10,20,Male,Never Used,11.69,US
4,100000,10,20,Male,Never Used,13.74,US
...,...,...,...,...,...,...,...
5378,350000,20,60,Female,Current user,526.82,US
5379,350000,20,60,Female,Current user,331.90,US
5380,350000,20,60,Female,Current user,526.82,US
5381,350000,20,60,Female,Current user,331.90,US


## Cleaning UK Dataset

In [ ]:
def main():
    # Setting UK_df as global to use across code
    global UK_df

    # Defining file name and path, and reading the CSV
    input_file = "final_UK_quotes.csv"
    output_path = Path.cwd().parent / "data" / "raw" / input_file  # Obtaining the directory
    UK_df = pd.read_csv(output_path)

    # Removing commas and "$" sign, then convert to float
    UK_df["Premium"] = UK_df["Premium"].replace({r"\£": "", ",": ""}, regex=True).astype(float)
    UK_df.rename(columns={"Premium": "Premium (£)"}, inplace=True) 

    # Adding "Country" column to differentiate after merging
    UK_df["Country"] = "UK"


    # Printing the DataFrame
    display(UK_df)
    

main()

,Coverage Amount,Term Length,Age,Gender,Is_Smoker,Premium (£),Country
0,100000,10,20,Male,Smoker,5.00,UK
1,100000,10,20,Male,Smoker,5.01,UK
2,100000,10,20,Male,Smoker,5.01,UK
3,100000,10,20,Male,Smoker,5.18,UK
4,100000,10,20,Male,Smoker,5.48,UK
...,...,...,...,...,...,...,...
8058,5000000,30,50,Female,Smoker,2591.49,UK
8059,5000000,30,50,Female,Smoker,2814.44,UK
8060,5000000,30,50,Female,Smoker,3510.62,UK
8061,5000000,30,50,Female,Smoker,4043.75,UK


## Merging US and UK Datasets

In [ ]:
def main():
    # Clean column names in both DataFrames
    UK_df.columns = UK_df.columns.str.strip()
    US_df.columns = US_df.columns.str.strip()

    quotes_df = pd.concat([US_df, UK_df], axis=0, ignore_index=True)

    file_name = "quotes.csv"
    # Define the file path (adjust the path as needed)
    output_path = Path.cwd().parent / "data" / "clean" / file_name

    # Renaming "Gender" to "Is_Male" and changing the values to 1/0 based on gender
    quotes_df.rename(columns={"Gender": "Is_Male"}, inplace=True)
    quotes_df["Is_Male"] = quotes_df["Is_Male"].apply(lambda x: 1 if x == "Male" else 0)

    # Changing the values in "Is_Smoker" column to 1/0 based on smoking status
    quotes_df["Is_Smoker"] = quotes_df["Is_Smoker"].apply(lambda x: 1 if x in ["Current user", "Smoker"] else 0)
    
    # Renaming "Country" to "Is_UK" and changing to binary variable
    quotes_df.rename(columns={"Country": "Is_UK"}, inplace=True)
    quotes_df["Is_UK"] = quotes_df["Is_UK"].apply(lambda x: 1 if x == "UK" else 0)

    # Displaying final DataFrame before saving to CSV
    display(quotes_df)

    # Saving DataFrame to CSV
    quotes_df.to_csv(output_path, index=False)
    print(f"📁 All done — saved {file_name} file to {output_path}")


main()

,Coverage Amount,Term Length,Age,Is_Male,Is_Smoker,Premium (£),Is_UK
0,100000,10,20,1,0,7.36,0
1,100000,10,20,1,0,7.95,0
2,100000,10,20,1,0,11.27,0
3,100000,10,20,1,0,11.69,0
4,100000,10,20,1,0,13.74,0
...,...,...,...,...,...,...,...
13441,5000000,30,50,0,1,2591.49,1
13442,5000000,30,50,0,1,2814.44,1
13443,5000000,30,50,0,1,3510.62,1
13444,5000000,30,50,0,1,4043.75,1


Coverage Amount      int64
Term Length          int64
Age                  int64
Is_Male              int64
Is_Smoker            int64
Premium (£)        float64
Is_UK                int64
dtype: object

📁 All done — saved quotes.csv file to c:\Users\Andrew\Documents\code_projects\Assignments\Y2\Empirical_Project\data\clean\quotes.csv
